# Bayesian Neural Network Example

From https://keras.io/examples/keras_recipes/bayesian_neural_networks/

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [6]:
# using the tensorflow wine dataset for this example

def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [5]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [7]:
# Creating model inputs
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


## Standard Neural Network

In [8]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling wine_quality-train.tfrecord...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to /home/tdo/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [32]:
type(test_dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [10]:
# train the baseline model with MSE

num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()

baseline_model = create_baseline_model()

run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 1s 21ms/step - loss: 30.3704 - root_mean_squared_error: 5.5109 - val_loss: 28.0471 - val_root_mean_squared_error: 5.2959
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 28.4859 - root_mean_squared_error: 5.3372 - val_loss: 26.4149 - val_root_mean_squared_error: 5.1395
Epoch 3/100
17/17 [==============================] - 0s 6ms/step - loss: 26.8911 - root_mean_squared_error: 5.1857 - val_loss: 24.8956 - val_root_mean_squared_error: 4.9895
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 25.3651 - root_mean_squared_error: 5.0364 - val_loss: 23.4674 - val_root_mean_squared_error: 4.8443
Epoch 5/100
17/17 [==============================] - 0s 6ms/step - loss: 23.8875 - root_mean_squared_error: 4.8875 - val_loss: 22.1195 - val_root_mean_squared_error: 4.7031
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 22.4533 - root_mean_squared_error

Epoch 96/100
17/17 [==============================] - 0s 6ms/step - loss: 0.5668 - root_mean_squared_error: 0.7528 - val_loss: 0.5619 - val_root_mean_squared_error: 0.7496
Epoch 97/100
17/17 [==============================] - 0s 6ms/step - loss: 0.5670 - root_mean_squared_error: 0.7530 - val_loss: 0.5608 - val_root_mean_squared_error: 0.7488
Epoch 98/100
17/17 [==============================] - 0s 6ms/step - loss: 0.5655 - root_mean_squared_error: 0.7520 - val_loss: 0.5595 - val_root_mean_squared_error: 0.7480
Epoch 99/100
17/17 [==============================] - 0s 6ms/step - loss: 0.5668 - root_mean_squared_error: 0.7529 - val_loss: 0.5588 - val_root_mean_squared_error: 0.7475
Epoch 100/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5633 - root_mean_squared_error: 0.7505 - val_loss: 0.5581 - val_root_mean_squared_error: 0.7470
Model training finished.
Train RMSE: 0.75
Evaluating model performance...
Test RMSE: 0.747


In [28]:
baseline_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
alcohol (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
chlorides (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
citric acid (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
density (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [13]:
# Check some sources. The model only gives point estimates
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.2 - Actual: 7.0
Predicted: 5.9 - Actual: 6.0
Predicted: 6.1 - Actual: 8.0
Predicted: 6.4 - Actual: 6.0
Predicted: 5.4 - Actual: 5.0
Predicted: 6.3 - Actual: 7.0
Predicted: 6.1 - Actual: 7.0
Predicted: 5.6 - Actual: 5.0
Predicted: 5.6 - Actual: 7.0
Predicted: 6.6 - Actual: 8.0


## Bayesian Neural Network

This network has probabilistic weights, so it will make a different prediction each time it's called. The prediction is still a point estimate though so to get the uncertainty, the outputs need to be sampled. 

In [14]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [15]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)
    
    # create hidden layers with weight uncertainties
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units =units,
            make_prior_fn = prior,
            make_posterior_fn = posterior,
            kl_weight = 1/ train_size,
            activation = 'sigmoid',
        )(features)
    
    # output is a single point estimate
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

In [16]:
# Train with a small subset of data

num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Epoch 1/500
5/5 [==============================] - 1s 90ms/step - loss: 40.4809 - root_mean_squared_error: 6.3615 - val_loss: 38.9522 - val_root_mean_squared_error: 6.2401
Epoch 2/500
5/5 [==============================] - 0s 16ms/step - loss: 39.2066 - root_mean_squared_error: 6.2605 - val_loss: 36.1685 - val_root_mean_squared_error: 6.0127
Epoch 3/500
5/5 [==============================] - 0s 16ms/step - loss: 38.0467 - root_mean_squared_error: 6.1670 - val_loss: 36.0287 - val_root_mean_squared_error: 6.0015
Epoch 4/500
5/5 [==============================] - 0s 15ms/step - loss: 40.5903 - root_mean_squared_error: 6.3700 - val_loss: 38.3824 - val_root_mean_squared_error: 6.1942
Epoch 5/500
5/5 [==============================] - 0s 15ms/step - loss: 36.6559 - root_mean_squared_error: 6.0532 - val_loss: 40.0413 - val_root_mean_squared_error: 6.3266
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 35.9877 - root_mean_squared_error: 5.997

5/5 [==============================] - 0s 15ms/step - loss: 10.2611 - root_mean_squared_error: 3.1998 - val_loss: 9.5084 - val_root_mean_squared_error: 3.0800
Epoch 96/500
5/5 [==============================] - 0s 14ms/step - loss: 9.4762 - root_mean_squared_error: 3.0752 - val_loss: 9.4523 - val_root_mean_squared_error: 3.0706
Epoch 97/500
5/5 [==============================] - 0s 14ms/step - loss: 10.8230 - root_mean_squared_error: 3.2867 - val_loss: 9.3667 - val_root_mean_squared_error: 3.0567
Epoch 98/500
5/5 [==============================] - 0s 15ms/step - loss: 9.2819 - root_mean_squared_error: 3.0432 - val_loss: 10.2794 - val_root_mean_squared_error: 3.2029
Epoch 99/500
5/5 [==============================] - 0s 16ms/step - loss: 9.4724 - root_mean_squared_error: 3.0747 - val_loss: 7.9668 - val_root_mean_squared_error: 2.8191
Epoch 100/500
5/5 [==============================] - 0s 15ms/step - loss: 9.7426 - root_mean_squared_error: 3.1182 - val_loss: 8.7060 - val_root_mean_squar

Epoch 143/500
5/5 [==============================] - 0s 15ms/step - loss: 3.1615 - root_mean_squared_error: 1.7699 - val_loss: 3.1404 - val_root_mean_squared_error: 1.7643
Epoch 144/500
5/5 [==============================] - 0s 15ms/step - loss: 3.4416 - root_mean_squared_error: 1.8485 - val_loss: 2.7638 - val_root_mean_squared_error: 1.6562
Epoch 145/500
5/5 [==============================] - 0s 15ms/step - loss: 2.3511 - root_mean_squared_error: 1.5239 - val_loss: 2.8031 - val_root_mean_squared_error: 1.6658
Epoch 146/500
5/5 [==============================] - 0s 14ms/step - loss: 3.0757 - root_mean_squared_error: 1.7467 - val_loss: 2.1581 - val_root_mean_squared_error: 1.4605
Epoch 147/500
5/5 [==============================] - 0s 15ms/step - loss: 2.5003 - root_mean_squared_error: 1.5739 - val_loss: 2.9588 - val_root_mean_squared_error: 1.7124
Epoch 148/500
5/5 [==============================] - 0s 15ms/step - loss: 2.7430 - root_mean_squared_error: 1.6476 - val_loss: 2.9708 - val_

Epoch 238/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7246 - root_mean_squared_error: 0.8328 - val_loss: 0.7792 - val_root_mean_squared_error: 0.8682
Epoch 239/500
5/5 [==============================] - 0s 16ms/step - loss: 0.8067 - root_mean_squared_error: 0.8828 - val_loss: 0.7769 - val_root_mean_squared_error: 0.8630
Epoch 240/500
5/5 [==============================] - 0s 16ms/step - loss: 0.8394 - root_mean_squared_error: 0.9003 - val_loss: 0.7961 - val_root_mean_squared_error: 0.8751
Epoch 241/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7706 - root_mean_squared_error: 0.8598 - val_loss: 0.7711 - val_root_mean_squared_error: 0.8639
Epoch 242/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7341 - root_mean_squared_error: 0.8408 - val_loss: 0.7750 - val_root_mean_squared_error: 0.8630
Epoch 243/500
5/5 [==============================] - 0s 14ms/step - loss: 0.7555 - root_mean_squared_error: 0.8529 - val_loss: 0.8156 - val_

Epoch 333/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7024 - root_mean_squared_error: 0.8232 - val_loss: 0.6742 - val_root_mean_squared_error: 0.8037
Epoch 334/500
5/5 [==============================] - 0s 14ms/step - loss: 0.7591 - root_mean_squared_error: 0.8574 - val_loss: 0.7352 - val_root_mean_squared_error: 0.8402
Epoch 335/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7324 - root_mean_squared_error: 0.8407 - val_loss: 0.7882 - val_root_mean_squared_error: 0.8735
Epoch 336/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7177 - root_mean_squared_error: 0.8302 - val_loss: 0.6818 - val_root_mean_squared_error: 0.8109
Epoch 337/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7083 - root_mean_squared_error: 0.8236 - val_loss: 0.7114 - val_root_mean_squared_error: 0.8238
Epoch 338/500
5/5 [==============================] - 0s 15ms/step - loss: 0.6905 - root_mean_squared_error: 0.8141 - val_loss: 0.7315 - val_

Epoch 428/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7010 - root_mean_squared_error: 0.8214 - val_loss: 0.7161 - val_root_mean_squared_error: 0.8288
Epoch 429/500
5/5 [==============================] - 0s 14ms/step - loss: 0.6559 - root_mean_squared_error: 0.7941 - val_loss: 0.6610 - val_root_mean_squared_error: 0.7957
Epoch 430/500
5/5 [==============================] - 0s 14ms/step - loss: 0.6805 - root_mean_squared_error: 0.8087 - val_loss: 0.6979 - val_root_mean_squared_error: 0.8193
Epoch 431/500
5/5 [==============================] - 0s 15ms/step - loss: 0.6672 - root_mean_squared_error: 0.7987 - val_loss: 0.6594 - val_root_mean_squared_error: 0.7921
Epoch 432/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7018 - root_mean_squared_error: 0.8208 - val_loss: 0.6637 - val_root_mean_squared_error: 0.7960
Epoch 433/500
5/5 [==============================] - 0s 15ms/step - loss: 0.6972 - root_mean_squared_error: 0.8170 - val_loss: 0.6727 - val_

In [29]:
bnn_model_small.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
fixed acidity (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
volatile acidity (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
citric acid (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
residual sugar (InputLayer)     [(None, 1)]          0                                            
____________________________________________________________________________________________

In [17]:
# call the outputs multiple times to sample from the predictions

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )




In [18]:
compute_predictions(bnn_model_small)

Predictions mean: 6.19, min: 5.82, max: 6.41, range: 0.59 - Actual: 7.0
Predictions mean: 5.81, min: 4.96, max: 6.25, range: 1.29 - Actual: 6.0
Predictions mean: 6.1, min: 5.77, max: 6.35, range: 0.58 - Actual: 8.0
Predictions mean: 6.28, min: 5.94, max: 6.48, range: 0.54 - Actual: 6.0
Predictions mean: 5.23, min: 4.44, max: 5.97, range: 1.53 - Actual: 5.0
Predictions mean: 6.21, min: 5.73, max: 6.45, range: 0.72 - Actual: 7.0
Predictions mean: 6.07, min: 5.74, max: 6.34, range: 0.6 - Actual: 7.0
Predictions mean: 5.51, min: 4.5, max: 6.18, range: 1.68 - Actual: 5.0
Predictions mean: 5.61, min: 4.79, max: 6.28, range: 1.49 - Actual: 7.0
Predictions mean: 6.34, min: 5.96, max: 6.51, range: 0.55 - Actual: 8.0


In [19]:
# train using the whole dataset

num_epochs= 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/500
17/17 [==============================] - 1s 24ms/step - loss: 38.4344 - root_mean_squared_error: 6.1992 - val_loss: 36.6167 - val_root_mean_squared_error: 6.0510
Epoch 2/500
17/17 [==============================] - 0s 6ms/step - loss: 38.2668 - root_mean_squared_error: 6.1857 - val_loss: 36.3791 - val_root_mean_squared_error: 6.0311
Epoch 3/500
17/17 [==============================] - 0s 6ms/step - loss: 35.6182 - root_mean_squared_error: 5.9677 - val_loss: 34.0362 - val_root_mean_squared_error: 5.8336
Epoch 4/500
17/17 [==============================] - 0s 6ms/step - loss: 33.0991 - root_mean_squared_error: 5.7528 - val_loss: 36.6831 - val_root_mean_squared_error: 6.0563
Epoch 5/500
17/17 [==============================] - 0s 6ms/step - loss: 32.6212 - root_mean_squared_error: 5.7111 - val_loss: 33.0169 - val_root_mean_squared_error: 5.7457
Epoch 6/500
17/17 [==============================] - 0s 6ms/step - loss: 31.1429 - root_mean_squared_error

Epoch 96/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6785 - root_mean_squared_error: 0.8177 - val_loss: 0.6866 - val_root_mean_squared_error: 0.8225
Epoch 97/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6498 - root_mean_squared_error: 0.7998 - val_loss: 0.6318 - val_root_mean_squared_error: 0.7876
Epoch 98/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6729 - root_mean_squared_error: 0.8142 - val_loss: 0.6964 - val_root_mean_squared_error: 0.8293
Epoch 99/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6810 - root_mean_squared_error: 0.8190 - val_loss: 0.6563 - val_root_mean_squared_error: 0.8040
Epoch 100/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6606 - root_mean_squared_error: 0.8064 - val_loss: 0.6380 - val_root_mean_squared_error: 0.7920
Epoch 101/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6683 - root_mean_squared_error: 0.8110 - val_loss: 0.7045 - va

17/17 [==============================] - 0s 6ms/step - loss: 0.6243 - root_mean_squared_error: 0.7827 - val_loss: 0.6054 - val_root_mean_squared_error: 0.7697
Epoch 191/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6222 - root_mean_squared_error: 0.7816 - val_loss: 0.6180 - val_root_mean_squared_error: 0.7786
Epoch 192/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6216 - root_mean_squared_error: 0.7812 - val_loss: 0.5872 - val_root_mean_squared_error: 0.7585
Epoch 193/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6200 - root_mean_squared_error: 0.7800 - val_loss: 0.5973 - val_root_mean_squared_error: 0.7655
Epoch 194/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6150 - root_mean_squared_error: 0.7767 - val_loss: 0.6377 - val_root_mean_squared_error: 0.7919
Epoch 195/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6319 - root_mean_squared_error: 0.7879 - val_loss: 0.6001 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6127 - root_mean_squared_error: 0.7743 - val_loss: 0.6014 - val_root_mean_squared_error: 0.7670
Epoch 285/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6208 - root_mean_squared_error: 0.7795 - val_loss: 0.6106 - val_root_mean_squared_error: 0.7731
Epoch 286/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6312 - root_mean_squared_error: 0.7864 - val_loss: 0.6135 - val_root_mean_squared_error: 0.7749
Epoch 287/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6063 - root_mean_squared_error: 0.7705 - val_loss: 0.5951 - val_root_mean_squared_error: 0.7626
Epoch 288/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6164 - root_mean_squared_error: 0.7769 - val_loss: 0.6169 - val_root_mean_squared_error: 0.7774
Epoch 289/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6146 - root_mean_squared_error: 0.7758 - val_loss: 0.6168 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.5984 - root_mean_squared_error: 0.7648 - val_loss: 0.5990 - val_root_mean_squared_error: 0.7641
Epoch 379/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6146 - root_mean_squared_error: 0.7745 - val_loss: 0.5897 - val_root_mean_squared_error: 0.7589
Epoch 380/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6007 - root_mean_squared_error: 0.7659 - val_loss: 0.6055 - val_root_mean_squared_error: 0.7687
Epoch 381/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6004 - root_mean_squared_error: 0.7658 - val_loss: 0.5999 - val_root_mean_squared_error: 0.7653
Epoch 382/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6024 - root_mean_squared_error: 0.7671 - val_loss: 0.6108 - val_root_mean_squared_error: 0.7735
Epoch 383/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6145 - root_mean_squared_error: 0.7747 - val_loss: 0.5988 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6004 - root_mean_squared_error: 0.7654 - val_loss: 0.5986 - val_root_mean_squared_error: 0.7636
Epoch 473/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6054 - root_mean_squared_error: 0.7692 - val_loss: 0.6130 - val_root_mean_squared_error: 0.7731
Epoch 474/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5936 - root_mean_squared_error: 0.7611 - val_loss: 0.5964 - val_root_mean_squared_error: 0.7622
Epoch 475/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6074 - root_mean_squared_error: 0.7699 - val_loss: 0.5928 - val_root_mean_squared_error: 0.7608
Epoch 476/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5949 - root_mean_squared_error: 0.7615 - val_loss: 0.5875 - val_root_mean_squared_error: 0.7577
Epoch 477/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6078 - root_mean_squared_error: 0.7702 - val_loss: 0.5989 - val_root_mea

In [20]:
# compute the predictions for using more data
compute_predictions(bnn_model_full)


Predictions mean: 6.23, min: 5.89, max: 6.47, range: 0.58 - Actual: 7.0
Predictions mean: 5.7, min: 5.29, max: 5.99, range: 0.7 - Actual: 6.0
Predictions mean: 6.08, min: 5.74, max: 6.36, range: 0.62 - Actual: 8.0
Predictions mean: 6.52, min: 6.32, max: 6.66, range: 0.34 - Actual: 6.0
Predictions mean: 5.26, min: 4.86, max: 5.97, range: 1.1 - Actual: 5.0
Predictions mean: 6.41, min: 5.99, max: 6.56, range: 0.58 - Actual: 7.0
Predictions mean: 6.07, min: 5.7, max: 6.34, range: 0.64 - Actual: 7.0
Predictions mean: 5.54, min: 4.96, max: 6.18, range: 1.22 - Actual: 5.0
Predictions mean: 5.69, min: 5.23, max: 6.18, range: 0.95 - Actual: 7.0
Predictions mean: 6.61, min: 6.44, max: 6.73, range: 0.29 - Actual: 8.0


## Modeling outputs as Probability Distributions

We can also have outputs as probability distributions

In [24]:
def create_probabilistic_bbn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)
    
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # create a probabilistic output Normal distribution and use the Dense layer to produce 
    # the parameters of the distributions.
    # we have units = 2 to learn both the mean and variance of the Normal distribution
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# we will use a negative log likelihood here instead of MSE
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


In [25]:
num_epochs = 1000
prob_bnn_model = create_probabilistic_bbn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 1s 27ms/step - loss: 543.2576 - root_mean_squared_error: 6.9060 - val_loss: 360.2090 - val_root_mean_squared_error: 6.6521
Epoch 2/1000
17/17 [==============================] - 0s 7ms/step - loss: 343.1700 - root_mean_squared_error: 6.6828 - val_loss: 305.7303 - val_root_mean_squared_error: 6.6411
Epoch 3/1000
17/17 [==============================] - 0s 7ms/step - loss: 257.8816 - root_mean_squared_error: 6.5605 - val_loss: 427.4093 - val_root_mean_squared_error: 6.7848
Epoch 4/1000
17/17 [==============================] - 0s 7ms/step - loss: 227.8298 - root_mean_squared_error: 6.5288 - val_loss: 447.2904 - val_root_mean_squared_error: 6.6370
Epoch 5/1000
17/17 [==============================] - 0s 6ms/step - loss: 202.8528 - root_mean_squared_error: 6.4236 - val_loss: 183.2293 - val_root_mean_squared_error: 6.5263
Epoch 6/1000
17/17 [==============================] - 0s 6ms/step - loss: 170.7900 - root_m

Epoch 48/1000
17/17 [==============================] - 0s 6ms/step - loss: 2.5741 - root_mean_squared_error: 3.7801 - val_loss: 2.5320 - val_root_mean_squared_error: 3.7547
Epoch 49/1000
17/17 [==============================] - 0s 7ms/step - loss: 2.4249 - root_mean_squared_error: 3.6809 - val_loss: 2.4960 - val_root_mean_squared_error: 3.8169
Epoch 50/1000
17/17 [==============================] - 0s 6ms/step - loss: 2.4219 - root_mean_squared_error: 3.6820 - val_loss: 2.3472 - val_root_mean_squared_error: 3.5392
Epoch 51/1000
17/17 [==============================] - 0s 6ms/step - loss: 2.3565 - root_mean_squared_error: 3.5426 - val_loss: 2.3225 - val_root_mean_squared_error: 3.5475
Epoch 52/1000
17/17 [==============================] - 0s 6ms/step - loss: 2.3589 - root_mean_squared_error: 3.6053 - val_loss: 2.2348 - val_root_mean_squared_error: 3.3339
Epoch 53/1000
17/17 [==============================] - 0s 7ms/step - loss: 2.2958 - root_mean_squared_error: 3.4472 - val_loss: 2.2867 

17/17 [==============================] - 0s 6ms/step - loss: 1.2083 - root_mean_squared_error: 1.1364 - val_loss: 1.2102 - val_root_mean_squared_error: 1.1497
Epoch 143/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1974 - root_mean_squared_error: 1.1424 - val_loss: 1.2001 - val_root_mean_squared_error: 1.1872
Epoch 144/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2192 - root_mean_squared_error: 1.1302 - val_loss: 1.2056 - val_root_mean_squared_error: 1.1411
Epoch 145/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2203 - root_mean_squared_error: 1.1445 - val_loss: 1.2013 - val_root_mean_squared_error: 1.1373
Epoch 146/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2149 - root_mean_squared_error: 1.1478 - val_loss: 1.1957 - val_root_mean_squared_error: 1.1387
Epoch 147/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2075 - root_mean_squared_error: 1.1555 - val_loss: 1.2063 - val_roo

17/17 [==============================] - 0s 7ms/step - loss: 1.1693 - root_mean_squared_error: 1.1014 - val_loss: 1.1910 - val_root_mean_squared_error: 1.1223
Epoch 237/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1810 - root_mean_squared_error: 1.1033 - val_loss: 1.2057 - val_root_mean_squared_error: 1.1165
Epoch 238/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1809 - root_mean_squared_error: 1.0891 - val_loss: 1.1731 - val_root_mean_squared_error: 1.1210
Epoch 239/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1809 - root_mean_squared_error: 1.0962 - val_loss: 1.1784 - val_root_mean_squared_error: 1.1207
Epoch 240/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1866 - root_mean_squared_error: 1.1177 - val_loss: 1.1653 - val_root_mean_squared_error: 1.0946
Epoch 241/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1847 - root_mean_squared_error: 1.0967 - val_loss: 1.1629 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1752 - root_mean_squared_error: 1.0998 - val_loss: 1.1666 - val_root_mean_squared_error: 1.1218
Epoch 331/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1682 - root_mean_squared_error: 1.0956 - val_loss: 1.1670 - val_root_mean_squared_error: 1.0939
Epoch 332/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1757 - root_mean_squared_error: 1.1014 - val_loss: 1.1608 - val_root_mean_squared_error: 1.1116
Epoch 333/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1754 - root_mean_squared_error: 1.0982 - val_loss: 1.1612 - val_root_mean_squared_error: 1.1093
Epoch 334/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1657 - root_mean_squared_error: 1.0902 - val_loss: 1.1802 - val_root_mean_squared_error: 1.1221
Epoch 335/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1692 - root_mean_squared_error: 1.0984 - val_loss: 1.1661 - val_roo

...

17/17 [==============================] - 0s 7ms/step - loss: 1.1746 - root_mean_squared_error: 1.0962 - val_loss: 1.1842 - val_root_mean_squared_error: 1.1114
Epoch 613/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1674 - root_mean_squared_error: 1.0857 - val_loss: 1.1500 - val_root_mean_squared_error: 1.0846
Epoch 614/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1605 - root_mean_squared_error: 1.0861 - val_loss: 1.1520 - val_root_mean_squared_error: 1.0526
Epoch 615/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1531 - root_mean_squared_error: 1.0679 - val_loss: 1.1679 - val_root_mean_squared_error: 1.1106
Epoch 616/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1581 - root_mean_squared_error: 1.0663 - val_loss: 1.1755 - val_root_mean_squared_error: 1.0792
Epoch 617/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1649 - root_mean_squared_error: 1.0876 - val_loss: 1.1523 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1641 - root_mean_squared_error: 1.0741 - val_loss: 1.1483 - val_root_mean_squared_error: 1.0453
Epoch 707/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1539 - root_mean_squared_error: 1.0829 - val_loss: 1.1907 - val_root_mean_squared_error: 1.0423
Epoch 708/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1565 - root_mean_squared_error: 1.0743 - val_loss: 1.1613 - val_root_mean_squared_error: 1.0516
Epoch 709/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1583 - root_mean_squared_error: 1.0728 - val_loss: 1.1727 - val_root_mean_squared_error: 1.0774
Epoch 710/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1580 - root_mean_squared_error: 1.0817 - val_loss: 1.1511 - val_root_mean_squared_error: 1.0730
Epoch 711/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1540 - root_mean_squared_error: 1.0777 - val_loss: 1.1456 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1468 - root_mean_squared_error: 1.0667 - val_loss: 1.1446 - val_root_mean_squared_error: 1.0490
Epoch 801/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1559 - root_mean_squared_error: 1.0632 - val_loss: 1.1541 - val_root_mean_squared_error: 1.0749
Epoch 802/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1581 - root_mean_squared_error: 1.0801 - val_loss: 1.1586 - val_root_mean_squared_error: 1.0423
Epoch 803/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1502 - root_mean_squared_error: 1.0785 - val_loss: 1.1631 - val_root_mean_squared_error: 1.0406
Epoch 804/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1610 - root_mean_squared_error: 1.0732 - val_loss: 1.1540 - val_root_mean_squared_error: 1.0402
Epoch 805/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1511 - root_mean_squared_error: 1.0712 - val_loss: 1.1506 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1534 - root_mean_squared_error: 1.0663 - val_loss: 1.1403 - val_root_mean_squared_error: 1.0859
Epoch 895/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1466 - root_mean_squared_error: 1.0653 - val_loss: 1.1523 - val_root_mean_squared_error: 1.0561
Epoch 896/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1501 - root_mean_squared_error: 1.0806 - val_loss: 1.1492 - val_root_mean_squared_error: 1.0989
Epoch 897/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1533 - root_mean_squared_error: 1.0874 - val_loss: 1.1597 - val_root_mean_squared_error: 1.1213
Epoch 898/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1476 - root_mean_squared_error: 1.0735 - val_loss: 1.1572 - val_root_mean_squared_error: 1.0728
Epoch 899/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1518 - root_mean_squared_error: 1.0780 - val_loss: 1.1597 - val_roo

17/17 [==============================] - 0s 7ms/step - loss: 1.1463 - root_mean_squared_error: 1.0763 - val_loss: 1.1528 - val_root_mean_squared_error: 1.0848
Epoch 989/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1468 - root_mean_squared_error: 1.0643 - val_loss: 1.1640 - val_root_mean_squared_error: 1.0872
Epoch 990/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1528 - root_mean_squared_error: 1.0683 - val_loss: 1.1461 - val_root_mean_squared_error: 1.0376
Epoch 991/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1530 - root_mean_squared_error: 1.0673 - val_loss: 1.1459 - val_root_mean_squared_error: 1.0612
Epoch 992/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1444 - root_mean_squared_error: 1.0484 - val_loss: 1.1509 - val_root_mean_squared_error: 1.0954
Epoch 993/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1541 - root_mean_squared_error: 1.0722 - val_loss: 1.1559 - val_roo

In [26]:
# the outputs now has the mean and standard deviation so we can compute the confidence intervals
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )


Prediction mean: 6.09, stddev: 0.77, 95% CI: [7.59 - 4.59] - Actual: 7.0
Prediction mean: 5.56, stddev: 0.68, 95% CI: [6.9 - 4.23] - Actual: 6.0
Prediction mean: 5.81, stddev: 0.7, 95% CI: [7.18 - 4.44] - Actual: 8.0
Prediction mean: 6.45, stddev: 0.82, 95% CI: [8.06 - 4.83] - Actual: 6.0
Prediction mean: 5.26, stddev: 0.64, 95% CI: [6.51 - 4.01] - Actual: 5.0
Prediction mean: 6.36, stddev: 0.81, 95% CI: [7.95 - 4.78] - Actual: 7.0
Prediction mean: 5.85, stddev: 0.71, 95% CI: [7.25 - 4.45] - Actual: 7.0
Prediction mean: 5.58, stddev: 0.68, 95% CI: [6.91 - 4.25] - Actual: 5.0
Prediction mean: 5.62, stddev: 0.69, 95% CI: [6.98 - 4.26] - Actual: 7.0
Prediction mean: 6.57, stddev: 0.84, 95% CI: [8.21 - 4.93] - Actual: 8.0


In [27]:
prob_bnn_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
fixed acidity (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
volatile acidity (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
citric acid (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
residual sugar (InputLayer)     [(None, 1)]          0                                            
____________________________________________________________________________________________